<a href="https://colab.research.google.com/github/niharika-sojitra/COMP-5800-Project/blob/main/SED_ORG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://towardsdatascience.com/training-t5-for-paraphrase-generation-ab3b5be151a2

https://github.com/hetpandya/paraphrase-datasets-pretrained-models/blob/main/examples/t5_paraphrase_model_training_example.ipynb

In [10]:
!pip install simpletransformers datasets tqdm pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 250 kB 7.5 MB/s 
     |████████████████████████████████| 451 kB 64.8 MB/s 
     |████████████████████████████████| 43 kB 2.4 MB/s 
     |████████████████████████████████| 10.3 MB 52.1 MB/s 
     |████████████████████████████████| 1.9 MB 67.8 MB/s 
     |████████████████████████████████| 182 kB 69.6 MB/s 
     |████████████████████████████████| 168 kB 61.1 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 168 kB 75.3 MB/s 
     |████████████████████████████████| 166 kB 71.3 MB/s 
     |████████████████████████████████| 166 kB 62.8 MB/s 
     |████████████████████████████████| 162 kB 73.8 MB/s 
     |████████████████████████████████| 162 kB 66.7 MB/s 
     |████████████████████████████████| 158 kB 74.5 MB/s 
     |████████████████████████████████| 157 kB 69.4 MB/s 
     |█████████████████████████████

In [2]:
!pip install transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 10.1 MB/s 
     |████████████████████████████████| 1.3 MB 26.6 MB/s 
     |████████████████████████████████| 182 kB 47.9 MB/s 
     |████████████████████████████████| 7.6 MB 3.0 MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pandas as pd
dataset_df = pd.read_csv('/content/drive/MyDrive/Project-5800/parallel_detoxification_dataset_small.tsv', sep='\t')
#df = pd.read_csv("https://github.com/s-nlp/parallel_detoxification_dataset/blob/main/parallel_detoxification_dataset_small.tsv", error_bad_lines=False)
#df = pd.read_csv('/content/drive/MyDrive/Project-5800/parallel.csv', error_bad_lines=False)
df.head()

,toxic_comment,civil_comment
0,. or the loud ass one - thousand ton beast roa...,or the loud one - thousand ton beast roaring ...
1,""" mandated "" and "" right fucking now "" would b...","""Mandated' and ""right now"" would be good."
2,""" mandated "" and "" right fucking now "" would b...","""mandated"" and"" right away"" would be good"
3,""" mandated "" and "" right fucking now "" would b...",mandated and right would be good
4,* neither * of my coworkers gave a shit when i...,Neither of my co-workers cared when it came to...


In [7]:
# Renaming the columns
dataset_df.columns = ["input_text","target_text"]
# Adding a prefix. Here we shall keep "paraphrase" as a prefix.
dataset_df["prefix"] = "paraphrase"

In [8]:
from sklearn.model_selection import train_test_split
train_data,test_data = train_test_split(dataset_df,test_size=0.1)

In [9]:
args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 256,
    "num_train_epochs": 4,
    "num_beams": None,
    "do_sample": True,
    "top_k": 50,
    "top_p": 0.95,
    "use_multiprocessing": False,
    "save_steps": -1,
    "save_eval_checkpoints": True,
    "evaluate_during_training": False,
    "adam_epsilon": 1e-08,
    "eval_batch_size": 6,
    "fp_16": False,
    "gradient_accumulation_steps": 16,
    "learning_rate": 0.0003,
    "max_grad_norm": 1.0,
    "n_gpu": 1,
    "seed": 42,
    "train_batch_size": 6,
    "warmup_steps": 0,
    "weight_decay": 0.0,
}


In [11]:
from simpletransformers.t5 import T5Model
import sklearn
model = T5Model("t5","t5-small", args=args)

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:174: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [12]:
model.train_model(train_data, eval_data=test_data, use_cuda=True,acc=sklearn.metrics.accuracy_score)

  0%|          | 0/2500 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3672: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3547: UserWarning: `as_target_tokenizer` is deprec

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/417 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/417 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/417 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/417 [00:00<?, ?it/s]

(104, 1.8049298403426433)

In [13]:
from simpletransformers.t5 import T5Model
import os
root_dir = os.getcwd()
trained_model_path = os.path.join(root_dir,"outputs")
args = {
"overwrite_output_dir": True,
"max_seq_length": 256,
"max_length": 50,
"top_k": 50,
"top_p": 0.95,
"num_return_sequences": 5
}
trained_model = T5Model("t5",trained_model_path,args=args)

In [19]:
prefix = "paraphrase"

In [21]:

pred = trained_model.predict([f"{prefix}: 'how freaking narcissistic do you have to be?'"])
print(pred)
#Output:


Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/5 [00:00<?, ?it/s]

[['You have to be narcissistic', "'how crazy do you have to be?'", "'how narcissistic do you have to be?'", 'how crazy you have to be', "'how narcissistic do you have to be?"]]
